# Exploring OpenAI V1 functionality

On 11.06.23 OpenAI released a number of new features, and along with it bumped their Python SDK to 1.0.0. This notebook shows off the new features and how to use them with LangChain.

In [ ]:
!pip install "openai>=1"

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage

## [Vision](https://platform.openai.com/docs/guides/vision)

OpenAI released multi-modal models, which can take a sequence of text and images as input.

In [2]:
chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=256)
chat.invoke(
    [
        HumanMessage(
            content=[
                {"type": "text", "text": "What is this image showing"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://python.langchain.com/assets/images/langchain_stack-da369071b058555da3d491a695651f15.jpg",
                        "detail": "auto",
                    },
                },
            ]
        )
    ]
)

AIMessage(content='The image appears to be a diagram representing the architecture or components of a software platform named "LangChain." This diagram outlines various layers and elements of the platform, which seems to be related to language processing or computational linguistics, as suggested by the context clues in the names of the components.\n\nHere\'s a breakdown of the components shown:\n\n- **LangSmith**: This seems to be a tool or suite related to testing, evaluation, monitoring, feedback, and annotation within the platform.\n\n- **LangServe**: This could represent a service layer that exposes the platform\'s capabilities as REST API endpoints.\n\n- **Templates**: These are likely reference applications provided as starting points or examples for users of the platform.\n\n- **Chains, agents, agent executors**: This section describes the common application logic, perhaps indicating that the platform uses a chain of agents or processes to execute tasks.\n\n- **Model I/O**: Thi

## [OpenAI assistants](https://platform.openai.com/docs/assistants/overview)

> The Assistants API allows you to build AI assistants within your own applications. An Assistant has instructions and can leverage models, tools, and knowledge to respond to user queries. The Assistants API currently supports three types of tools: Code Interpreter, Retrieval, and Function calling

In [1]:
from langchain_experimental.openai_assistant.base import OpenAIAssistantRunnable


In [2]:
interpreter_assistant = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview"
)
output = interpreter_assistant.invoke({"content": "What's 10 - 4 raised to the 2.7"})
output

[ThreadMessage(id='msg_bG50eUX5DhIKSTLKV7EgaHMM', assistant_id='asst_p9qkGkw1FfYEd90HLvPalaox', content=[MessageContentText(text=Text(annotations=[], value='The value of \\( 10 - 4^{2.7} \\) is approximately -32.224.'), type='text')], created_at=1699379844, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_2cc89e4VYnxBOhPQsR5pgcPE', thread_id='thread_ugmxDTcETSsM52k4NBsRB6tR')]

### As a LangChain agent with arbitrary tools

In [9]:
from langchain.tools import E2BDataAnalysisTool

tools = [E2BDataAnalysisTool(api_key="...")]

In [5]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True
)

#### Using AgentExecutor


In [7]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke({"content": "What's 10 - 4 raised to the 2.7"})

{'content': "What's 10 - 4 raised to the 2.7",
 'output': 'The result of \\( 10 - 4^{2.7} \\) is approximately \\(-32.224\\).'}

#### Custom execution

In [10]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True
)

In [12]:
from langchain.schema.agent import AgentFinish

tool_map = {tool.name: tool for tool in tools}
response = agent.invoke({"content": "What's 10 - 4 raised to the 2.7"})
while not isinstance(response, AgentFinish):
    tool_outputs = []
    for action in response:
        tool_output = tool_map[action.tool].invoke(action.tool_input)
        print(action.tool, action.tool_input, tool_output, end="\n\n")
        tool_outputs.append({"output": tool_output, "tool_call_id": action.tool_call_id})
    response = agent.invoke({"tool_outputs": tool_outputs, "run_id": action.run_id, "thread_id": action.thread_id})
    
print(response)

e2b_data_analysis {'python_code': "result = 10 - 4**2.7\nprint('The result of 10 - 4 raised to the 2.7 is:', result)"} {"stdout": "The result of 10 - 4 raised to the 2.7 is: -32.22425314473263", "stderr": "", "artifacts": []}

return_values={'output': 'The result of \\(10 - 4\\) raised to the \\(2.7\\) is approximately \\(-32.22425314473263\\).'} log='' run_id='run_ORbUDQ9DELkjjPcxP4arCoBA' thread_id='thread_Kaou0OV59KmtQpqR9e49jt8q'


## [JSON mode](https://platform.openai.com/docs/guides/text-generation/json-mode)

Constrain the model to only generate valid JSON. Note that you must include a system message with instructions to use JSON for this mode to work.

Only works with certain models. 

In [8]:
chat = ChatOpenAI(model="gpt-3.5-turbo-1106").bind(
    response_format={"type": "json_object"}
)

output = chat.invoke(
    [
        SystemMessage(
            content="Extract the 'name' and 'origin' of any companies mentioned in the following statement. Return a JSON list."
        ),
        HumanMessage(
            content="Google was founded in the USA, while Deepmind was founded in the UK"
        ),
    ]
)
print(output.content)

{
  "companies": [
    {
      "name": "Google",
      "origin": "USA"
    },
    {
      "name": "Deepmind",
      "origin": "UK"
    }
  ]
}


In [9]:
import json

json.loads(output.content)

{'companies': [{'name': 'Google', 'origin': 'USA'},
  {'name': 'Deepmind', 'origin': 'UK'}]}

## [System fingerprint](https://platform.openai.com/docs/guides/text-generation/reproducible-outputs)

OpenAI sometimes changes model configurations in a way that impacts outputs. Whenever this happens, the system_fingerprint associated with a generation will change.

In [2]:
chat = ChatOpenAI(model="gpt-3.5-turbo-1106")
output = chat.generate(
    [
        [
            SystemMessage(
                content="Extract the 'name' and 'origin' of any companies mentioned in the following statement. Return a JSON list."
            ),
            HumanMessage(
                content="Google was founded in the USA, while Deepmind was founded in the UK"
            ),
        ]
    ]
)
print(output.llm_output)

{'token_usage': {'completion_tokens': 43, 'prompt_tokens': 49, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_eeff13170a'}
